In [1]:
import numpy as np
from typing import List, Tuple


# Introducción

El método de divide y vencerás es una estrategia poderosa para diseñar algoritmos asintóticamente eficientes. Conoces varios algoritmos tipo *divide y vencerás*, eg., `merge_sort()`. 

Recuerde que para divide y vencerás, resuelves un problema dado (instancia) de forma recursiva.
Si el problema es lo suficientemente pequeño, el **caso base** se resuelve directamente sin recurrencia. Sino se ejecuta la recursión. La recursión consta de tres fases: 

   * **Divide**. Divide el problema original en varios subproblemas

   * **Conquista**. Los problemas se resuelven recursivamente

   * **Combina**. Combina las soluciones de los subproblemas para formar una solución al problema original.


Por tanto un algoritmo *divide y vencerás* descompone un gran problema en subproblemas más pequeños, que a su vez pueden descomponerse en subproblemas aún más pequeños, y así sucesivamente. La recursión toca fondo cuando alcanza el caso base y el subproblema es lo suficientemente pequeño como para resolverlo directamente sin recurrencia adicional.
Es importante insistir en que los problemas deben ser disjuntos, i.e., que no se solapen.



# Multiplicación de matrices


## Algoritmo iterativo:

* Suponga dos matrices $A$ y $B$ de dimensión $n \times n$ cuyos elementos son $(a_{ij})$ y $(b_{ij})$ respectivamente. El producto $A \cdot B$ será $ c_{ij} = \sum_k a_{ik} \cdot b_{kj}$.
El siguiente código implementa la multiplicación de matrices conforme la ecuación anterior 

In [19]:
def square_matrix_multiply (a:np.array, b:np.array)->np.array:
    '''.....'''
    nrows, ncols, interm = a.shape[0], b.shape[1], b.shape[0]
    
    c = np.zeros((nrows, ncols ))
    
    for i in range (nrows):
        for j in range(ncols):
            for k in range (interm):
                c[i,j] += a[i, k]*b[k, j]
    return(c)

#####--- Driver program

a = np.array([[1, 0], [0, 1]])
b = np.array([[4, 1], [2, 2]])
c = square_matrix_multiply (a,b)

print(c) 


[[4. 1.]
 [2. 2.]]


Inicializar la matriz $c$ tiene coste $\Theta(n^2)$.  Cada uno de los bucles triplemente anidados anteriores se itera exactamente $n$ veces, lo que significa que el algoritmo anterior tiene una complejidad de tiempo de ejecución de $\Theta (n^3)$.

¿Podemos hacerlo mejor? ¿Podríamos multiplicar dos matrices en tiempo inferior a $n^3$? 
>
>You might at first think that any matrix multiplication algorithm must take $\omega(n^3)$ time, since the natural definition of matrix multiplication requires that many multiplications. You would be incorrect, however: we have a way to multiply matrices in $o(n^3)$ time. Strassen’s remarkable recursive algorithm for multiplying $n$ by $n$ matrices runs in $\Theta(n^{\log7})$ time.

## Algoritmo recursivo: 

Supongamos que la dimensión de las matrices $n \times n$ es una potencia exacta de 2 (m = $2^n$).
Esta suposición nos permite dividir la matriz en bloques o cuadrantes más pequeños de tamaño $m/2 \times m/2$ y nos asegura que la dimensión sea un número entero. 
Este proceso se denomina **block partitioning**.
Los bloques se comportan como si fueran elementos atómicos y el producto de $A$ y $B$ se puede expresar en función del producto de sus cuadrantes. 

Cuando *partimos* las matrices $A,B$ y $C$ en bloques de igual tamaño, obtenemos las matrices

\begin{equation} 
A={\begin{bmatrix}A_{00}&A_{01}\\A_{10}&A_{11}\end{bmatrix}},
\quad B={\begin{bmatrix}B_{00}&B_{01}\\B_{10}&B_{11}\end{bmatrix}},
\quad C={\begin{bmatrix}C_{00}&C_{01}\\C_{10}&C_{11}\end{bmatrix}},\quad
\end{equation}

con $A_{ij},B_{ij},C_{ij} \in \text{Mat}_{2^{n-1} \times 2^{n-1}}$ 
El algoritmo *naive* expresamos C como el producto de los cuadrantes de $A$ y $B$:

\begin{equation}
{\begin{bmatrix}C_{00}&C_{01}\\C_{10}&C_{11}\end{bmatrix}}={\begin{bmatrix}A_{00}B_{00}+A_{01}B_{10}&A_{00}B_{01}+A_{01}B_{11}\\
A_{10}B_{00}+A_{11}B_{10}&A_{10}B_{01}+A_{11}B_{11}\end{bmatrix}}.
\end{equation}


**Importante** Esta estrategia nos permite un enfoque recursivo: podemos seguir particionando las matrices en cuadrantes hasta que sean lo suficientemente pequeñas como para multiplicarlas de forma sencilla. 

<div class="alert-success">

¿Cómo transformar las ecuaciones anteriores en pseudocódigo (o código real)? 
Existen dos enfoques comunes para implementar la partición de matriz. 

* Una estrategia consiste en asignar almacenamiento temporal para mantener las cuatro submatrices de $A$,  $A_{11}, A_{12}, A_{21}$ y $A_{22}$ y las cuatro submatrices $B_{11}, B_{12}, B_{21}$ y $B_{22}$ de $B$. A continuación se debe copiar cada elemento de $A$ y $B$ en su ubicación correspondiente en la submatriz apropiada. Después realizar los productos correspondientes (paso recursivo de *conquista*) y, finalmente, copiar los elementos en cada una de las cuatro submatrices $C_{11}, C_{12}, C_{21}$ y $C_{22}$ en sus ubicaciones correspondientes en $C$. 
En esta estrategia simplemente el coste de *distribuir* adecuadamente los datos entre las matrices y sus submatrices correspondientes es $\Theta(n^2)$, ya que se copian $3n^2$ elementos. 
* El segundo enfoque trabaja con los índices de las matrices. Al no tener que reservarse memoria ni copiarse los datos es más rápido y práctico. Las submatrices se pueden especificar indicando en qué parte de la matriz se encuentran, sin necesidad de *mover* ningún elemento. Por tanto, particionar una matriz (o recursivamente, una submatriz) solo implica realizar operaciones aritmeticas sobre las ubicaciones. Estas operaciones tiene un coste constante, no dependen del tamaño de la matriz, es decir $\Theta(1)$. Finalmente, los cambios que hay que efectuar en los elementos de las submatrices de $C$ actualizan la matriz $C$ original, ya que ocupan el mismo almacenamiento.

Nosotros optaremos por el segundo planteamiento **(Cormen et al, sección 4.1)**.

</div>


```python
matrix_multiply_recursive (A:matrix, B:matrix, C:matrix, n):
    ''' Pseudocódigo '''
    # Base case
    if  n == 1: 
        C[0, 0]  = C[0, 0] + A[0,0] * B[0,0]
        return
    
    # Divide. 
    A = partition (A)
    B = partition (B)
    C = partition (C)
    
    # Conquer. 
    matrix_multiply_recursive (A[0, 0], B[0,0], C[0,0],n-1)
    matrix_multiply_recursive (A[0, 0], B[0,1], C[0,1],n-1)
    matrix_multiply_recursive (A[1, 0], B[0,0], C[1,0],n-1)
    matrix_multiply_recursive (A[1, 0], B[1,0], C[1,1],n-1)
    matrix_multiply_recursive (A[0, 1], B[1,0], C[0,0],n-1)
    matrix_multiply_recursive (A[0, 1], B[1,1], C[1,2],n-1)
    matrix_multiply_recursive (A[1, 1], B[1,0], C[1,0],n-1)
    matrix_multiply_recursive (A[1, 1], B[1,1], C[1,1],n-1)
```


**Implementación algoritmo**

* Una implementación eficiente del pseudocódigo anterior requiere gestionar adecuadamente la memoria. Se debería evitar reservar memoria en las llamadas recursivas. En la implementación que se muestra a continuación las particiones de las matrices son *vistas* de los arrays que se han reservado en el *frame principal*.

In [3]:
def partition (m:np.array, n:int)-> List[np.array]:
    ''' Devuelve las submatrices bloque de dimension n/2 x n/2'''
    dim = 2**(n-1)
    
    m_11 = m[0:dim, 0:dim]
    m_12 = m[0:dim, dim:]
    m_21 = m[dim:, 0:dim]
    m_22 = m[dim:, dim:]
    
    return (m_11, m_12, m_21, m_22)


def  matrix_multiply_recursive (a:np.array, b:np.array, c:np.array, n:int, row=0, col=0):
    ''' IMPORTANT only valid for matrix with  dim=2**n x 2**n'''
    
    # Base case
    if  n == 0: 
        c[row,col] += a[0,0] * b[0,0]
        return

    # Divide. 
    a_00, a_01, a_10, a_11 = partition (a, n)
    b_00, b_01, b_10, b_11 = partition (b, n)

    # Conquer. 
    dim = 2**(n-1)
    matrix_multiply_recursive (a_00, b_00, c, n-1, row, col)
    matrix_multiply_recursive (a_00, b_01, c, n-1, row, col+dim)
    matrix_multiply_recursive (a_10, b_00, c, n-1, row+dim, col)
    matrix_multiply_recursive (a_10, b_01, c, n-1, row+dim, col+dim)
    matrix_multiply_recursive (a_01, b_10, c, n-1, row, col)
    matrix_multiply_recursive (a_01, b_11, c, n-1, row, col+dim)
    matrix_multiply_recursive (a_11, b_10, c, n-1, row+dim, col)
    matrix_multiply_recursive (a_11, b_11, c, n-1, row+dim, col+dim)
    
    return

##### Driver program

# for reproducibility
np.random.seed(123)
n = 2
a = np.random.rand (2**n, 2**n)
b = np.random.rand (2**n, 2**n)

c = np.zeros((2**n, 2**n))
matrix_multiply_recursive (a, b, c, n)
print(c)

[[0.63444316 0.78638275 0.71035541 0.8361271 ]
 [1.309407   1.23448993 1.10686201 1.16204531]
 [0.79858465 0.98830279 0.73101863 0.88989051]
 [0.62221958 0.72184999 0.48833857 0.68063776]]


**Coste**

* En el *caso base* $(n=0)$ se realiza una multiplicación y una adición $\rightarrow$ $\Theta(1)$.

* El caso recursivo ocurre cuando $n > 1$. Al usar aritmética de índice para dividir las matrices, el coste de las particiones es $\Theta(1)$. A continuación, se invoca recursivamente al procedimiento `matrix_multiply_recursive()`un total de ocho veces sobre matrices de tamaño $\frac{n}{2} \times \frac{n}{2}$.
Las primeras cuatro llamadas recursivas calculan los primeros términos de las ecuaciones, y las siguientes cuatro llamadas calculan y suman los segundos términos. Cada llamada recursiva agrega el producto de una submatriz de A y una submatriz de B a la submatriz apropiada de $C$ *in-place*. El coste de cada una de las 8 llamadas recursiva es $T(n/2)$. Notad como en este algoritmo no hay fase de *combinación* ya que la matriz $C$ se actualiza, tal y como hemos visto,  *in-place*.
Por tanto el tiempo total para la recursión 

\begin{equation}
T(n) = 8 T(n/2) + \Theta(1)
\end{equation}

* La ley de recurrencia puede resolverse por el método maestro (ver apéndice) anterior obteniendo un coste $O(n^3)$. Como puedes observar **no** hemos mejorado el coste del algoritmo iterativo inicial de multiplicación de matrices.

## Algoritmo de Strassen:

Suponga que las matrices $A, B$ y $C$ se dividen en matrices por bloques como en el algoritmo recursivo anterior. El algoritmo de Strassen define las nuevas matrices:

\begin{aligned}
M_{1}&=(A_{11}+A_{22})(B_{11}+B_{22});\\
M_{2}&=(A_{21}+A_{22})B_{11};\\
M_{3}&=A_{11}(B_{12}-B_{22});\\
M_{4}&=A_{22}(B_{21}-B_{11});\\
M_{5}&=(A_{11}+A_{12})B_{22};\\
M_{6}&=(A_{21}-A_{11})(B_{11}+B_{12});\\
M_{7}&=(A_{12}-A_{22})(B_{21}+B_{22}),\\
\end{aligned}

usando solo 7 multiplicaciones (una para cada $M_{k}$) en lugar de 8. Ahora podemos expresar el $C_{ij}$ en términos de $M_{k}$:

\begin{equation}
 \begin{bmatrix}C_{11}&C_{12}\\C_{21}&C_{22}\end{bmatrix}={\begin{bmatrix}M_{1}+M_{4}-M_{5}+M_{7}&M_{3}+M_{5}\\M_{2}+M_{4}&M_{1}-M_{2}+M_{3}+M_{6}\end{bmatrix}}
\end{equation}

**Implementación:**

In [16]:
def split(matrix:np.array, current_size)->List:
    """
    Splits a given matrix into quarters.
    Input: nxn matrix
    Output: tuple containing 4 n/2 x n/2 matrices corresponding to a, b, c, d
    """
    
    current_new_size = current_size//2
    
    a11 = matrix[0:current_new_size,0:current_new_size] # top left
    a12 = matrix[0:current_new_size,current_new_size:current_size] # top right
    a21 = matrix[current_new_size:current_size, 0:current_new_size] # bottom left
    a22 = matrix[current_new_size:current_size, current_new_size:current_size] # bottom right
    
    return a11, a12, a21, a22

def strassen(a:np.array, b:np.array, threshold = 1)->np.array:
    """
    From: https://towardsdatascience.com/understanding-deepmind-and-strassen-algorithms-9bdb3d8b6ea6
    
    Computes matrix product by divide and conquer approach, recursively.
    Input: nxn matrices a and b
           threshold: If the dimension is below a given threshold (or not divisible by 2) 
           compute the remaining product with standard numpy
    
    Output: nxn matrix, product of a and a
    """
    
    current_size = len(a)
   
    # Base case. Assign threshold = 1 when size of matrices is 1x1
    if current_size <= threshold:
        c = np.matmul (a, b)
        return c

    # Splitting the matrices into quadrants. This will be done recursively
    # until the base case is reached.
    a11, a12, a21, a22 = split(a, current_size)
    b11, b12, b21, b22 = split(b, current_size)

    # Computing the 7 products, recursively (m1, m2...m7)
    
    a_ = np.add(a11, a22)
    b_ = np.add(b11, b22) 
    m1 = strassen(a_, b_) # iterate over the first multiplication
    
    a_ = np.add(a21, a22) 
    m2 = strassen(a_, b11) # second product 
    
    b_ = np.subtract(b12, b22) 
    m3 = strassen(a11, b_) # third product 
    
    b_ = np.subtract(b21, b11)
    m4 = strassen(a22, b_) # fourth product 
    
    a_ = np.add(a11, a12) 
    m5 = strassen(a_, b22) # fifth product 
    
    a_ = np.subtract(a21, a11) 
    b_ = np.add(b11, b12) 
    m6 = strassen(a_, b_) # sixth product 
    
    a_ = np.subtract(a12, a22) 
    b_ = np.add(b21, b22) 
    m7 = strassen(a_, b_) # seventh product
    
    
    # compute the c element for the product matrix 
    
    c12 = np.add(m3, m5) 
    c21 = np.add(m2, m4) 
    
    a_ = np.add(m1, m4)
    b_ = np.add(a_, m7)
    c11 = np.subtract(b_, m5)
    
    a_ = np.add(m1, m3) 
    b_ = np.add(a_, m6) 
    c22 = np.subtract(b_, m2)
    

    # Combining the 4 quadrants into a single matrix
    # return the final matrix 
    
    c = np.zeros([current_size, current_size])
    
    current_new_size = current_size // 2
    
    c[0:current_new_size, 0:current_new_size] = c11 # top left
    c[0:current_new_size,current_new_size:current_size] = c12 # top right
    c[current_new_size:current_size, 0:current_new_size] = c21 # bottom left
    c[current_new_size:current_size, current_new_size:current_size] = c22 # bottom righ
    
    return c

##### Driver program

# for reproducibility
np.random.seed(123)

n = 2
a = np.random.rand (2**n, 2**n)
b = np.random.rand (2**n, 2**n)

c = strassen (a, b, threshold = 1)

print(c)


[[0.63444316 0.78638275 0.71035541 0.8361271 ]
 [1.309407   1.23448993 1.10686201 1.16204531]
 [0.79858465 0.98830279 0.73101863 0.88989051]
 [0.62221958 0.72184999 0.48833857 0.68063776]]


¿Podrías mejorar la implementación del algoritmo?

**Coste y rendimiento:**

* E el código se realizan las siguientes acciones:
 1. Particionar las matrices de entrada $a$ y $b$  de dimensión $n \times n $ en matrices de dimensión $n/2 \times n/2$. Al igual que en `matrix_multiply_recursive` ni se genera memoria ni se copian datos. Se accede a las particiones mediante operación algebraica. Coste: $\Theta(1)$.
 2. Generar $n/2 \times n/2$ matrices $S_1, S_2, \ldots, S_{10}$ con las sumas o diferencias parciales (indicadas en el código con las variables `a_` y `b_`),  $\Theta(n^2)$
 3. Multiplicar recursivamente 7 veces matrices de dimensión  $n/2 \times n/2$  para obtener matrices  de dimensión $n/2 \times n/2$.  Coste: $7 \cdot T(n/2)$.
 4. Sumar o restar matrices   $n/2 \times n/2$  8 veces (indicadas en el código con las variables `à_` y `b_`),  $\Theta(n^2)$ 
 5. Generar las matrices $n \times n$ y copiar en ella los datos, $\Theta(n^2)$ 
 
  Sumando todos los costes, se obtiene la ley de recurrencia

\begin{equation}
T(n) = 7 T(n/2) +O(n^2)
\end{equation}

* Esta recurrencia se resuelve fácilmente por el método maestro (ver apéndice). El algoritmo de Strassen es asintóticamente más rápido $O(n^{\log 7})= O(n^{2.81})$ que el procedimiento básico de multiplicación de matrices. 


* Se suelen mencionar algunas de las siguientes cuatro razones por las que el algoritmo de Strassen **no** siempre es  método elegido para la multiplicación de matrices:
>
>* *The constant factor hidden in the $O(n^{\log 7})$ running time of Strassen’s algorithm is larger than the constant factor in the $ O(n^3)$
`square_matrix_multiply()` procedure*.
>* *When the matrices are sparse, methods tailored for sparse matrices are faster*.
>* *Strassen’s algorithm is not quite as numerically stable as the regular approach. In other words, because of the limited precision of computer arithmetic on noninteger values, larger errors accumulate in Strassen’s algorithm than in `square_matrix_multiply()`*.
>* *The submatrices formed at the levels of recursion consume space*.

* Sin embargo, aunque el algoritmo de Strassen es inestable numéricamente para algunas aplicaciones, está dentro de los límites aceptables para otras muchas. En la práctica, las implementaciones rápidas de multiplicación de matrices para matrices densas usan el algoritmo de Strassen para tamaños de matriz por encima de un umbral y cambian a un método más simple una vez que el tamaño del subproblema se reduce por debajo del umbral. El valor exacto del umbral depende en gran medida del sistema. Se encontró que los umbrales considerando varios si stemas oscilan entre 400 y 2150.

* **Nota:** ¿Que podemos hacer cuando la dimensión de las matrices no sea potencia de 2? (consulta el apartado *consideraciones de implementación* del algoritmo de Strassen en Wikipedia)

**Para saber mas sobre la historia... y el presente**
 
El siguinte texto está extraído del Cormen et al (4 Edition), 
 
>"*Strassen’s algorithm  caused much excitement when it appeared in 1969.
Before then, few imagined the possibility of an algorithm asymptotically faster than
the basic MATRIX -MULTIPLY procedure. Shortly thereafter, S. Winograd reduced
the number of submatrix additions from 18 to 15 while still using seven submatrix
multiplications. 
This improvement, which Winograd apparently never published (and which is frequently miscited in the literature), may enhance the practicality of the method, but it does not affect its asymptotic performance. Probert [368]
described Winograd’s algorithm and showed that with seven multiplications, 15 additions is the minimum possible.
Strassen’s ‚ $O(n^{\log 7}) =  O( n^{2.81})$ bound for matrix multiplication held until 1987,
when Coppersmith and Winograd [103] made a significant advance, improving the  bound to $O(n^{2.376})$ time with a mathematically sophisticated but wildly impractical algorithm based on tensor products.
It took approximately 25 years before the asymptotic upper bound was again improved. 
In 2012 Vassilevska Williams [445] improved it to $O(n^{2.37287})$ and two years later Le Gall [278] achieved $O(n^{2.372859639})$, both of them using mathematically fascinating but impractical algorithms. 
The best lower bound to date is just the obvious $\Omega (n^2)$ bound (obvious because any algorithm for matrix multiplication must fill in the $n^2$ elements of the product matrix)*."

Sin embargo, ..., enero 2021 $O(n^{2.3728596})$, septiembre 2022 entra IA en juego[1], octubre 2022 la revancha de los humanos, ...
 
[1] [Discovering faster matrix multiplication algorithms with reinforcement learning](https://www.nature.com/articles/s41586-022-05172-4) y para [entender la idea la nota divulgativa emitida por DeepMind](https://www.deepmind.com/blog/discovering-novel-algorithms-with-alphatensor) 

**Ejercicios** Importantes los problemas

# Multiplicación de números

_**Referencia:**_  Weiis, M. Allan, Chapter 10.2.4

Suponga un número $x$ en una determinada base $B$. Expresemos $x$ en potencias de $B$ y supongamos que por sencillez que el número de coeficientes es par
\begin{aligned}
x & = a_{n-1} B^{n-1} + \ldots + a_{n/2} B^{n/2} + \ldots + a_0  = \\
   &=  \left( a_{n-1} B^{n/2 -1} + \ldots + a_{n/2} \right) B^{n/2} + a_{n/2-1} B^{n/2-1} + \ldots + a_0 = \\
   & = x_1 B^{n/2} + x_0
\end{aligned}  

por tanto $x_0 \sim B^{n/2-1} < B^{n/2}$

Si $y$ es otro número cualquiera, $y  = y_1 B^{n/2} + y_0$, podemos expresar el producto  $x \cdot y$  

\begin{aligned}
x \cdot y&=(x_{1}B^{n/2}+x_{0})(y_{1}B^{n/2}+y_{0})\\
&=x_{1}y_{1}B^{n}+(x_{1}y_{0}+x_{0}y_{1})B^{n/2}+x_{0}y_{0}\\
&=z_{2}B^{n}+z_{1}B^{n/2}+z_{0},\\
\end{aligned}
 
 donde

\begin{aligned}
   z_{2} &=x_{1} \cdot y_{1}, \\
   z_{1} &=x_{1} \cdot y_{0}+x_{0} \cdot y_{1}, \\
   z_{0} &=x_{0} \cdot y_{0}.
\end{aligned}

* Lo cual sugiere un algoritmo tipo *divide y vencerás* para multiplicar dos números $x$ e $y$. Descomponer $x$ e $y$ en los números mas pequeños $(x_0, x_1, y_0, y_1)$ cada uno con aproximádemente la mitad de dígitos que $x$ e $y$, efectuar las multiplicaciones recursivas y combinar los resultados según las fórmulas anteriores. 
* Al requerirse **cuatro multiplicaciones**, su coste sería

$$
T(n) = 4 T(n/2) + \Theta(1)
$$

* Si resolvemos la recurrencia anterior, obtenemos que el coste del algoritmo recursivo de multiplicación de dos números es $O(n^2)$. No hemos ganado nada sobre el coste del algoritmo tradicional de multiplicación de dos números. 

## Algoritmo de Karatsuba

* Sin embargo, Karatsuba observó que el producto $x \cdot y$ se puede realizar en solo **tres multiplicaciones** si se incrementa el número de adiciones y se expresa $z_1$ como

\begin{equation}
z_{1}=(x_{1}+x_{0})(y_{1}+y_{0})-z_{2}-z_{0}.
\end{equation}

* El algoritmo de Karatsuba es del tipo *divide y vencerás*. Permite calcular el producto de dos números grandes $x \cdot y$ usando tres multiplicaciones de números más pequeños, cada uno con aproximadamente la mitad de dígitos que $x$ o $y$, a costa de incrementar el número de adiciones.

* Para completar el algoritmo, debemos tener un caso base, que se pueda resolver sin recursividad. Cuando ambos números son de un dígito, podemos hacer la multiplicación por búsqueda en una tabla. Si un número tuviese cero dígitos, devolveríamos cero. En la práctica, si tuviéramos que usar este algoritmo, de debería elegir el caso base que fuese el más conveniente para la máquina. 

* Aunque este algoritmo tiene un mejor rendimiento asintótico que el algoritmo cuadrático estándar, rara vez se usa, porque para $n$ pequeño la sobrecarga es significativo, y para $n$ más grandes hay algoritmos aún mejores.

**Implementación**

In [3]:
def karatsuba(x: int, y: int) -> int:
    if x < 10 or y < 10:
        return x * y
    else:
        if x > y:
            x, y = y, x

        m = len(str(x))//2

        x1, x0 = divmod(x, 10**m)
        y1, y0 = divmod(y, 10**m)

        z2 = karatsuba(x1, y1)
        z0 = karatsuba(x0, y0)

        z1 = karatsuba(x1+x0, y1+y0) 

        return z2*10**(2*m) + (z1-z2-z0)*10**m + z0

#####----- Driver program

f1 = 12345
f2 = 6789

c = karatsuba(f1, f2)
print (c)

83810205


**Coste**

Teniendo en cuenta que

$$
T(n) = 3 T (n/2) + \Theta(1)
$$

Aplicando el método maestro  que $O(n^{\log 3}) = O(n^{1.59})$, ver apéndice.

# El problema $K$-Selection




* Dada una tabla $S$ de tamaño $n=|S|$ y un índice $k$ , se pretende devolver el valor del elemento que ocupa la $k–$ésima posición en una ordenación de $S$.
* Un caso particular del problema anterior es hallar la mediana de una secuencia de números. En este caso $k= \lceil n/2 \rceil$
* Solución más simple: ordenar $S$ y devolver el elemento que ocupa la posición $S[k-1]$ de la tabla, asumiendo que el rango de los índices de la tabla es $0:n-1$.
 * Su coste es $O(n  \cdot \log n)$, demasiado grande si tenemos que encontrar por ejemplo los vaĺores máximos ($k = n$) o mínimos ($k = 1$), para lo cual los algoritmos estándar que recorren el array tienen un coste lineal $\Theta(n)$
* Solución que utiliza un max-Heap con los $k$ menores elementos de la tabla (práctica 1 del curso). En este caso el coste es $O (n \cdot \log k)$  

*  Se puede utilizar la metodología de *analysis adversary* para mostrar que el problema de la selección tiene una cota inferior: cualquier algoritmo debe tener un costo $\Omega(n)$

<div class="alert-success">

**Pregunta:** ¿Podemos resolver el problema de la selección general en tiempo lineal? En esta sección veremos el algoritmo  quickselection() que es una variante del algoritmo de ordenación quicksort()
</div>

## Algoritmo quicksort()

Antes de describir un algoritmo con coste lineal para seleccionar el $k-$ésimo elemento de la ordenación de una tabla, recordaremos el algoritmo `quicksort()` para ordenar una tabla de elementos $S$.

* Quicksort es un algoritmo tipo *divide y vencerás*.
* La implementación más común de `quicksort()` **no requiere memoria adicional**, es, por tanto, una algoritmo que se puede ejecutar *in-place*. 

Quicksort consta, al menos, de los siguientes cuatros pasos, 

1. Si el número de elementos en la tabla $S$ fuese 0 o 1, entonces la tabla ya se encuentra ordenada. Caso base de la recursión.
2. Se debe elegir un elemento $v \in S$. A este elemento se denomina **pivote**.
3. Se deben distribuir los elementos restantes de $S$, es decir, $S−\{v \}$,  en dos grupos separados por el pivote $v$ tal que $S_1=\{ x \in S−{v} | x \leq v \}$, y $S_2=\{ x \in S−{v} | x \geq v \}$
4. Invocar a  $\text{quicksort}(S_1)$ seguido de  $\text{quicksort}(S_2)$


* Dado que el paso de partición describe ambiguamente qué hacer con los elementos de $S$ iguales al pivote, esto se convierte en una decisión de diseño. Una buena implementación del algoritmo manejará estos elementos de la manera más eficiente posible. Intuitivamente, esperaríamos que aproximadamente la mitad de los elementos iguales al pivote se incluyesen en $S_1$ y la otra mitad a $S_2$.

* Debe quedar claro que este algoritmo funciona; en cada ejecución del paso 3 se *ordena* el pivote. Sin emebargo,  **no** es evidente porqué este algoritmo más rápido que mergesort. Al igual que mergesort, resuelve recursivamente dos subproblemas y requiere trabajo lineal adicional (paso 3) para *dividir* el problema original en dos subproblemas. Sin embargo, a diferencia de mergesort, no se garantiza que los subproblemas sean del mismo tamaño, lo que es potencialmente **no** deseable. 

 La razón por la que quicksort es más rápido es porqué el paso de partición se puede realizar *in place*  de manera muy eficiente. Esta eficiencia compensa con creces la ausencia de llamadas recursivas sobre problemas del igual tamaño. 
 
* En el algoritmo tal y omo lo hemos descrito descrito hemos obviado algunos detalles muy importantes, por ejemplo, como elegir el pivote mas adecuado, o incorporar un *cutoff* (umbral) que restrinja la aplicación de quicksort a tablas con tamaño superior al *cutoff*. 
* Hay muchas formas de implementar los pasos 2 y 3; No todas son igual de eficientes. Pequeñas variaciones pueden producir drásticas modificaciones en el rendimiento del algoritmo.

* Como se puede intuir, el rendimiento del algoritmo dependerá de la posición en la que termine el pivote elegido.
 * En el **mejor caso**, el pivote termina en el centro de la lista, dividiéndola en dos sublistas de igual tamaño. En este caso, el orden de complejidad del algoritmo es $O(n \cdot \log n)$ como puedes comprobar fácilmente resolviendo la recurrencia con el **método maestro** 
 
 $$
 T(n) = 2 \cdot T \left( \frac{n}{2} \right) + O(n)
 $$
 
 * En el **peor caso**, el pivote termina en un extremo de la lista. Una sublista está vacía y la otra acoge a todos los elementos. El peor caso dependerá de la implementación del algoritmo, aunque habitualmente ocurre en listas que se encuentran ordenadas, o casi ordenadas. Si, por ejemplo, el algoritmo toma como pivote siempre el último (primer) elemento del array $S$, y el array estuviese ya ordenado, siempre va a generar a su derecha (izquierda) un array vacío, lo que es ineficiente. En este caso el orden de complejidad del algoritmo es $O(n^2)$, igual que los algoritmos *insertion sort* y *selection sort*.
 
 $$
 T(n) = T(0)+T(n-1) + O(n)= T(n-1) + O(n).
 $$
 
 * En el **caso promedio** las dos listas están *equilibradas*, el coste es $O(n\cdot \log n).$ (Cormen et al, Capítulo 7)

**Implementación de quicksort**

* Seguimos el planteamiento original de Hoare que crea las particiones $S_1$ y $S_2$ utilizando dos índices que recorren el array desde sus dos extremos (puedes consultar los detalles en la versión española de [wikipedia](https://es.wikipedia.org/wiki/Quicksort))
* **Nota**. En la función `split()` se ha elegido como pivote el **último elemento de la tabla**. En las transparencias del curso se elige el **primer elemento** de la tabla. 

In [38]:
def split(data:List, left:int, right:int)->int:
    ''' Hoare partition (adapted from Weiss)
    
    left is the left-most index of the List.
    right is the right-most index of the List.'''
    
    # Modify if other pivot is used. 
    index_pivot = right
    pivot = data[index_pivot]
    
    lIndex = left-1    
    rIndex = index_pivot
    
    while True:
        
        lIndex +=1
        while data[lIndex] < pivot :
            lIndex += 1
        
        rIndex -= 1
        while data[rIndex] > pivot:
            rIndex -= 1
        
        if lIndex < rIndex:
            data[lIndex], data[rIndex] = data[rIndex], data[lIndex]  
        
        else:
            break
     
    # put the pivot between the two sub-lists
    data[lIndex], data[right] = data[right], data[lIndex]  
    
    # return the position of the pivot in the array data
    return lIndex    
           
        

def quickSort(data, left, right):  
    if right <= left:    
        return
    
    index_pivot = split (data, left, right)
    print (f'Index_pivot:{index_pivot}, pivot:{data[index_pivot]} data:{data}')
    
    quickSort(data, left, index_pivot - 1)
    quickSort(data, index_pivot + 1, right)    
    return 


#------ Driver program

data = [40, 10, 100, 90, 20, 80, 50, 70, 60, 30]
before_sort = data[:]

# Execute quiksort
quickSort(data, 0, len(data)-1)
print(f'\nOrdered data:{data}')

#Check results, compare element by element 
print(f'Correct:{sorted(before_sort) == data}')

Index_pivot:2, pivot:30 data:[20, 10, 30, 90, 40, 80, 50, 70, 60, 100]
Index_pivot:0, pivot:10 data:[10, 20, 30, 90, 40, 80, 50, 70, 60, 100]
Index_pivot:9, pivot:100 data:[10, 20, 30, 90, 40, 80, 50, 70, 60, 100]
Index_pivot:5, pivot:60 data:[10, 20, 30, 50, 40, 60, 90, 70, 80, 100]
Index_pivot:3, pivot:40 data:[10, 20, 30, 40, 50, 60, 90, 70, 80, 100]
Index_pivot:7, pivot:80 data:[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

Ordered data:[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Correct:True


**Experimenta:**

En la función quicksort des-comenta la sentencia `print` para seguir la evolución paso a paso del algoritmo. Es importante que verifiques porqué funciona el algoritmo; Como después de cada llamada a la función split, el pivote se sitúa en su posición correcta y cúal es la secuencia en que se *ordenan* las particiones de la tabla.

* ¿Cómo podríamos mejorar el rendimiento del algoritmo en el peor de los casos? 
Para ello deberíamos asegurar que en cada partición las dos sub-listas permanezcan equilibradas. ¿Se te ocurra algún método? 
* ¿Que significa incorporar un *cutoff*?¿Cómo hacerlo?


**Mejorando quicksort:**


## Algoritmo quickselect()

* El algoritmo `quickselect()` básico utiliza la misma estrategia recursiva que quicksort para generar las particiones $S_1$ y $S_2$. 

 * Suponiendo que el tamaño de la lista $|S|$ es superior a un umbral (*cutoff*) donde los elementos simplemente se ordenan, se elige un elemento $v$, conocido como pivote.

  * Los elementos restantes de $S$ se colocan en dos particiones, $S_1$ y $S_2$.  La partición $S_1$ contendrá los elementos de $S$ que no sean mayores que $v$, y $S_2$ los elementos que no sean menores que $v$.

* Si $k=|S_1|+1$, entonces el pivote es el $k-$ésimo elemento más pequeño de $S$. Sino, si  $k \leq |S_1|$, entonces el $k-$ésimo elemento más pequeño se obtendrá,  recursivamente, en el $k-$ésimo elemento más pequeño de $S_1$. De lo contrario si $k > |S_1|+1$, entonces el pivote es el $k-$ésimo elemento más pequeño de $S_2$. 

* La principal diferencia entre quickselect y quicksort es que solo hay un subproblema para resolver en lugar de dos como ocurría en quicksort.

**Implementación de quickselect in-place**

* Para seguir la evolución del algoritmo des-comenta el print.
* En el siguiente enlace de [pythontutor](https://pythontutor.com/render.html#code=def%20split%28data,%20left,%20right%29%3A%0A%20%20%20%20'''%20Hoare%20partition%20%28adapted%20from%20Weiss%29%0A%20%20%20%20%0A%20%20%20%20left%20is%20the%20left-most%20index%20of%20the%20List.%0A%20%20%20%20right%20is%20the%20right-most%20index%20of%20the%20List.'''%0A%20%20%20%20%0A%20%20%20%20%23%20Modify%20if%20other%20pivot%20is%20used.%20%0A%20%20%20%20index_pivot%20%3D%20right%0A%20%20%20%20pivot%20%3D%20data%5Bindex_pivot%5D%0A%20%20%20%20%0A%20%20%20%20lIndex%20%3D%20left-1%20%20%20%20%0A%20%20%20%20rIndex%20%3D%20index_pivot%0A%20%20%20%20%0A%20%20%20%20while%20True%3A%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20lIndex%20%2B%3D1%0A%20%20%20%20%20%20%20%20while%20data%5BlIndex%5D%20%3C%20pivot%20%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20lIndex%20%2B%3D%201%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20rIndex%20-%3D%201%0A%20%20%20%20%20%20%20%20while%20data%5BrIndex%5D%20%3E%20pivot%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20rIndex%20-%3D%201%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20if%20lIndex%20%3C%20rIndex%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20data%5BlIndex%5D,%20data%5BrIndex%5D%20%3D%20data%5BrIndex%5D,%20data%5BlIndex%5D%20%20%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20break%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%23%20put%20the%20pivot%20between%20the%20two%20sub-lists%0A%20%20%20%20data%5BlIndex%5D,%20data%5Bright%5D%20%3D%20data%5Bright%5D,%20data%5BlIndex%5D%20%20%0A%20%20%20%20%0A%20%20%20%20%23%20return%20the%20position%20of%20the%20pivot%20in%20the%20array%20data%0A%20%20%20%20return%20lIndex%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%0A%0Adef%20quickSelect%28data,%20left,%20right,%20k%29%3A%0A%20%20%20%20'''%20Versi%C3%B3n%20in-place.%0A%20%20%20%20No%20usa%20cutoff.%20No%20controla%20excepciones'''%0A%20%20%20%20%0A%20%20%20%20%23%20check%20the%20list%20length%0A%20%20%20%20if%20%20%28right%20-%20left%29%20%3C%200%3A%20%20%20%20%0A%20%20%20%20%20%20%20%20return%0A%20%20%20%20%0A%20%20%20%20index%20%3D%20split%20%28data,%20left,%20right%29%0A%20%20%20%20%0A%20%20%20%20%23%20Comentar%20si%20no%20se%20quisiese%20seguir%20la%20evoluci%C3%B3n%20del%20algoritmo%20paso%20a%20paso%0A%20%20%20%20%23print%20%28f'index%3A%7Bindex%7D,%20k%3A%7Bk%7D,%20%7Bdata%7D'%29%0A%20%20%20%20%0A%20%20%20%20if%20%20k%20%3D%3D%20%28index%20%2B%201%29%20%20%3A%0A%20%20%20%20%20%20%20%20return%20data%5Bindex%5D%0A%20%20%20%20%0A%20%20%20%20elif%20k%20%3C%20%28index%20%2B%201%29%20%20%3A%0A%20%20%20%20%20%20%20%20return%20quickSelect%20%28data,%20left,%20index-1,%20k%29%0A%20%20%20%20%0A%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20return%20quickSelect%20%28data,%20index%2B1,%20right,%20k%20%29%20%20%20%20%0A%20%20%20%20%0Adata%20%3D%20%5B10,%20100,%201,%2090,%2030,%205,%2050,%2060%5D%0A%0Ak%3D5%0Aprint%28k,%20quickSelect%28data,%200,%20len%28data%29-1,%20k%29%29&cumulative=false&curInstr=98&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)
también puedes seguir la evolución del algoritmo paso a paso.

In [7]:
def quickSelect(data:List, left:int, right:int, k:k):
    ''' Versión in-place.
    No usa cutoff. No controla excepciones'''
    
    # check the list length
    if  (right - left) < 0:    
        return
    
    index = split (data, left, right)
    
    # Comentar si no se quisiese seguir la evolución del algoritmo paso a paso
    #print (f'index:{index}, k:{k}, {data}')
    
    if  k == (index + 1)  :
        return data[index]
    
    elif k < (index + 1)  :
        return quickSelect (data, left, index-1, k)
    
    else:
        return quickSelect (data, index+1, right, k )    
    
    
#---- Driver programm

data = [40, 30, 100, 90, 20, 80, 50, 70, 60, 10]

# run quickselect for all possible k-values
for k in range(1, len(data)+1):
    value = quickSelect (data, 0, len(data)-1, k)
    print(f'k:{k}, Value:{value}')

# run quicksort to verify the results
quickSort(data)
print (f'\n Ordered data: {data}')

k:1, Value:10
k:2, Value:20
k:3, Value:30
k:4, Value:40
k:5, Value:50
k:6, Value:60
k:7, Value:70
k:8, Value:80
k:9, Value:90
k:10, Value:100

 Ordered data: [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


**Implementación de quickselect _not in-place_**

Compara la implementación anterior de con la facilitada en las transparencias (se incluye el código a continuación). Observa:

* La versión de las transparencias es muy *ilustrativa*. El código Python refleja de manaera muy transparente la lógica del algoritmo, tanto de la función `split_not()` que realiza las particiones $S_1$ y $S_2$ como del propio algoritmo en si `quickseelct` 

* La implementación que se hace en `quickSelect_not()` a diferencia de la de `quickSelect()` **no** es *in-place*; ya que reserva memoria adicional para las sub-tablas $s_1$ y $s_2$. Ejecuta el enlace de [pythontutor](https://pythontutor.com/render.html#code=def%20split_not%20%28data%29%3A%0A%20%20%20%20'''%20Not%20in-place'''%0A%20%20%20%20%0A%20%20%20%20pivot_index%20%3D%20len%28data%29-1%0A%20%20%20%20pivot_value%20%3D%20data%5Bpivot_index%5D%20%0A%20%20%20%20%0A%20%20%20%20s_1%20%3D%20%5Bx%20for%20x%20in%20data%5B%3A-1%5D%20if%20x%20%3C%3D%20pivot_value%5D%0A%20%20%20%20s_2%20%3D%20%5Bx%20for%20x%20in%20data%20if%20x%20%3E%20pivot_value%5D%0A%0A%20%20%20%20return%20%28s_1,%20pivot_value,%20s_2%29%0A%0A%0Adef%20quickSelect_not%20%28data,%20k%29%3A%0A%20%20%20%20'''Not%20in-place'''%0A%20%20%20%20if%20len%28data%29%20%3C%201%3A%0A%20%20%20%20%20%20%20%20return%0A%20%20%20%20%0A%20%20%20%20s_1,%20pivot_value,%20s_2%20%3D%20split_not%20%28data%29%0A%20%20%20%20%0A%20%20%20%20pivot_index%20%3D%20len%28s_1%29%0A%20%20%20%20print%20%28f'index%3A%7Bpivot_index%7D,%20k%3A%7Bk%7D,%20data%3A%20%7Bs_1%7D%20,%20%7Bpivot_value%7D,%20%20%7Bs_2%7D'%29%0A%0A%20%20%20%20%0A%20%20%20%20if%20%28k-1%29%20%3D%3D%20pivot_index%20%3A%0A%20%20%20%20%20%20%20%20return%20pivot_value%0A%20%20%20%20elif%20%28k-1%29%20%3C%20pivot_index%20%3A%0A%20%20%20%20%20%20%20%20return%20quickSelect_not%20%28s_1,%20k%29%0A%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20return%20quickSelect_not%20%28s_2,%20k-%28pivot_index%2B1%29%29%0A%20%20%20%20%20%20%20%20%0A%20%23.---%20Driver%20program%0A%20%20%20%0Adata%20%3D%20%5B10,%20100,%201,%2090,%2030,%205,%2050,%2060%5D%0A%0Ak%3D5%0Aprint%28k,%20quickSelect_not%28data,%20k%29%29&cumulative=false&curInstr=34&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false) 
*paso a paso* para apreciar las diferencias en la gestión de la memoria entre ambas implementaciones. 

* Además para generar las particiones $S_1$ y $S_2$,  `split_not()`requiere iterar dos veces la tabla, una para hallar los elementos menores que el pivote y otra para los mayores.

* Presta atención a la diferencia en los argumentos de la segunda llamada en `quickSelect()` y en `quickSelect_not()`, es una consecuencia las diferencias entre ambas implementciones de la gestión de memoria.

In [17]:
def split_not (data:List):
    ''' Not in-place'''
    
    pivot_index = len(data)-1
    pivot_value = data[pivot_index] 
    
    s_1 = [x for x in data[:-1] if x <= pivot_value]
    s_2 = [x for x in data if x > pivot_value]

    return (s_1, pivot_value, s_2)

def quickSelect_not (data:List, k) -> int:
    '''Not in-place'''
    if len(data) < 1:
        return
    
    s_1, pivot_value, s_2 = split_not (data)
    
    pivot_index = len(s_1)
    
    # Comentar si no se quisiese seguir la evolución del algoritmo paso a paso
    print (f'index:{pivot_index}, k:{k}, data: {s_1} , {pivot_value},  {s_2}')

    if (k-1) == pivot_index :
        return pivot_value
    elif (k-1) < pivot_index :
        return quickSelect_not (s_1, k)
    else:
        return quickSelect_not (s_2, k-(pivot_index+1))
        
 #.--- Driver program
   
data = [10, 100, 20, 90, 30, 80, 50, 60, 40, 70, 5, 1]

k=5
print(k, quickSelect_not(data, k))

#for k in range(1, len(data)+1):
#    v = quickSelect_not(data, k)
#    print (k, v)
print(sorted(data))

index:0, k:5, data: [] , 1,  [10, 100, 20, 90, 30, 80, 50, 60, 40, 70, 5]
index:0, k:4, data: [] , 5,  [10, 100, 20, 90, 30, 80, 50, 60, 40, 70]
index:6, k:3, data: [10, 20, 30, 50, 60, 40] , 70,  [100, 90, 80]
index:3, k:3, data: [10, 20, 30] , 40,  [50, 60]
index:2, k:3, data: [10, 20] , 30,  []
5 30
[1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


**Análisis formal**

Al igual que ocurría en quicksort la eficiencia del algoritmo dependerá del tamaño de las particiones $S_1$ y $S_2$

* En el **mejor de los casos** las dos particiones en las sucesivas llamadas recursivas tienen el mismo tamaño

$$
T(n) = T(n/2) + O(n)
$$

ya que ahora, a diferencia de queicksort, tan solo se ejecuta una llamada recursiva.
Por el **método maestro** se obtiene inmediatamente ($a=1$, $b=2$ y $k=1$) que el coste es lineal $O(n)$ 

* En el **peor de los casos**, tabla ordenada y selección del pivote en un extremo de las sub-tablas, al igual que ocurría en quicksort el coste es cuadrático $O(N^2)$

* El **caso promedio**, el mas complicado de demostrar, el coste es lineal O(n) (Ver trasparenecias)


## Algoritmo QSelect5()  o de la *Mediana de las medianas*:

**Referencia**. Weiss A. Chapter 10.3.2 p.475 (4 Ed.)

<div class="alert-success">

 QSelect5 selecciona el pivote adecuado de forma que el coste en el caso peor de los casos sea lineal. 
 El pivote seleccionado logra que los tamaños de las sub-tablas sean *equivalentes*.
 El pivote es una *estimacón* de la mediana de los elementos de la tabla.
</div>

* El algoritmo básico para **seleccionar** el pivote en QSelect5 es el siguiente:

1. Distribuir los $|S|=n$ elementos en $ \Big\lfloor \frac{|S|}{5} \Big\rfloor$ grupos de cinco elementos cada uno, desechando el último grupo grupos sino tuviese cinco elementos (como máximo se desecharán cuatro elementos). 
 
2. Hallar la mediana de cada uno de los grupos.La mediana de cada uno de los grupos se puede obtener, por ejemplo, ordenando los elementos de cada grupo con mergesort. En ese caso para cada grupo se necesitarían 8 comparaciones. Formar una tabla con las medianas de todos los grupo $S' = [m_1, m_2, \ldots, m_{2W+1} ]$.

3. Encontrar la mediana de la tabla de las medianas $S'$ . Para hallarla podemos invocar a QSelect sobre la lista con las medianas. Develver la *mediana de las medianas* como el pivote, $v$. 

**Ejemplo**

Sea la lista $ S = [ 15, 3, 7, 2, 12, 9, 1, 6, 14, 11, 4, 8, 13, 5, 10 ] $ suponga que se ejecuta `QSelect5()` para seleccionar el  $k-$ésimo elemento en una ordenación de $S$. Describir paso a paso la evolución del algoritmo para hallar el valor del elemento que ocuparía el sexto lugar en una ordenación de $S$, i.e., cuando $k=6$.

>Asumiremos que el algoritmo `QSelect5()` incluye un *cutoff*. Es decir, que cuando QSelect se ejecute sobre una tabla con un tamaño inferior al *cutoff* el problema se resolverá ordenando los elementos (por ejemplo, llamando a mergesort) y eligiendo el elemento que ocupa la $k-$ésima posición. 
>
>1. La lista original se divide en $p = \lfloor |S|  / 5 \rfloor$  grupos de 5 elementos cada uno. En este ejemplo, $p = \lfloor 15 / 5  \rfloor= 3$ se crearían los tres grupos
$(15, 3, 7, 2, 12), (9, 1, 6, 14, 11), (4, 8, 13, 5, 10)$ 
>
>1. Hallar la mediana de cada uno de los grupos. Crear una nueva tabla con las medianas. En el ejemplo la lista de las medianas estará formada por los elementos  $T = [7, 9, 8]$ 
>
>1. Hallar la mediana de $T$, i.e.  la *mediana de las medianas*. Para ello  invocar a `QSelect5()`con $T$ y $k= \lceil |m|/2 \rceil$ como argumentos. En el ejemplo anterior se ejecutaría `Select5 (T, 2)` y se obtendría como mediana de las medianas el valor $v = 8$.
>
>1. A continuación, utilizar $v$ como pivote de $S$ para obtener las particiones $S_1$ y $S_2$ de la lista original $S$. En este ejemplo obtendríamos las particiones,
>
$$
\begin{aligned}
S_1 &= 3, 7, 2, 1, 6, 4, 5  \\ 
S_2 &= 15, 12, 9, 14, 11, 13, 10 
\end{aligned}
$$
>
>1. Como $k-1 = 5 < 7 $, siendo 7 el índice del pivote (cuyo valor es 8) en $\{ S_1 \cup 8 \cup S_2 \}$, nos quedaríamos con la partición $S_1$.
>1. Se vuelve a ejecutar el paso 1 en este caso sobre $S_1$
> * (paso 1) Se crea únicamente el grupo $(3,7,2,1,6)$ desechando el grupo $(4, 5)$ por tener menos de 5 elementos. 
> * (pasos 2 y 3) Se devuelve la *mediana de las medianas*. En este caso $v=3$
> * Se utiliza $v$ como pivote para hallar las nuevas particiones 
$$
\begin{aligned}
S_1 &= 2, 1 \\ 
S_2 &= 7, 6, 4, 5
\end{aligned}
$$
> * Al ser $k$ mayor que el índice del pivote nos quedamos con $S_2$. 
> * Como $|S_2| = 4 \leq5 $, el tamaño de $S_2$ es menor que el *cutoff*=5, `QSelect5()` devuelve la posición k-1 después de la ordenación de $S_2$   

**Implementación:**

_**Importante:** 
(1) El propósito de la implementación que se muestra a continuación es mostrar la lógica del algoritmo QSelect5 y permitirnos analizar la evolución del mismo paso a paso. Sin embargo **no** debe considerarse una implementación funcionalmente correcta al no satisfacerse muchos de los requisitos funcionales que deberían exigirse a una implementación correcta (e.g., gestión de elementos repetidos, gestión *in-place* de la memoria que realiza las particiones, gestión de excepciones,...etc).
(2) Para hallar el valor de la $k-$ésima posición de una tabla cuyo tamaño es inferior al *cutoff* se ha utilizado la función `sorted()`de Python en vez de mergesort tal y como se indica en la descripción del algoritmo._

In [41]:
def pivot (data:List, group_size, cutoff):
    '''Calculate the medians of medians'''

    n_group = len(data) // group_size
    index_median = group_size // 2 
    sublists =  [data[j:j+ group_size] for j in range(0, len(data), group_size)][:n_group]
    medians = [sorted(sub)[index_median] for sub in sublists]
    
    if len(medians) <= cutoff:
         pivot = sorted(medians)[len(medians)//2]
    else:
        pivot = QSelect5 (medians, len(medians)//2)
        
    return pivot
    
def partition (data, pivot): 
    '''Not valid with repeated elements. Not in-place'''
    s_1 = [x for x in data if x < pivot]
    s_2 = [x for x in data if x > pivot]
    
    return (s_1, pivot, s_2)
   
    
def QSelect5 (data:List, k:int, group_size=5, cutoff=5) -> int:
    ''' No admite elementos repetidos
    Importante: cutoff >= group_size'''
    
    # Caso base (con cutoff)
    if len(data) <= cutoff:
        return sorted(data)[k-1]
    
    # Obtiene el pivote (mediana de las medianas)
    pivot_value = pivot (data, group_size, cutoff)
    
    # Obtiene las particiones utilizando el pivote 
    s_1, pivot_value, s_2 = partition (data, pivot_value)

    pivot_index = len(s_1)
    #print (f'index:{pivot_index}, k:{k}, data: {s_1} , {pivot_value},  {s_2}')

    if (k-1) == pivot_index  :
        return pivot_value
    elif (k-1) < pivot_index :
        return QSelect5 (s_1, k)
    else:
        return QSelect5 (s_2, k-pivot_index-1)
    
#------ Driver program   

# Crea una lista de tamaño n con datos aleatorios
np.random.seed(123)
n = 1000
data = list(np.random.permutation(np.arange(0,n))[:n])

k = 6
print(f'k:{k}, value:{QSelect5(data, k)}')


#k-first sorted(data) values to check 
print (sorted(data)[:k])


k:6, value:5
[0, 1, 2, 3, 4, 5]


**Ejercicio:** 

Compara la evolución del algoritmo QSelect5() y qselect() sobre la lista $S = [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]$ para $k=6$. Para ello debes des-comentar la sentencia print en la implementación de algoritmos. ¿Qué conclusiones extraes respecto al tamaño de las sub-tablas? 

In [31]:
S = [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
k= 6

# Evolución con QSelect5()
print('Evolucion QSelect5()')
print(f'k:{k}, value:{QSelect5(S, k)}')

# Evolucion con quickSelect_not()
print('\nEvolucion quickselect()')
print(f'k:{k}, value={quickSelect_not(S, k)}')

Evolucion QSelect5()
index:7, k:6, data: [1, 5, 10, 20, 30, 40, 50] , 60,  [70, 80, 90, 100]
index:2, k:6, data: [1, 5] , 10,  [20, 30, 40, 50]
k:6, value:40

Evolucion quickselect()
index:11, k:6, data: [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90] , 100,  []
index:10, k:6, data: [1, 5, 10, 20, 30, 40, 50, 60, 70, 80] , 90,  []
index:9, k:6, data: [1, 5, 10, 20, 30, 40, 50, 60, 70] , 80,  []
index:8, k:6, data: [1, 5, 10, 20, 30, 40, 50, 60] , 70,  []
index:7, k:6, data: [1, 5, 10, 20, 30, 40, 50] , 60,  []
index:6, k:6, data: [1, 5, 10, 20, 30, 40] , 50,  []
index:5, k:6, data: [1, 5, 10, 20, 30] , 40,  []
k:6, value=40


### Costes QSelect5()

**Obteniendo el tamaño máximo de la mayor de las sub-tablas:**

Para que el algoritmo QSelect5 siga teniendo tenga un coste lineal también en el peor de los casos, debemos garantizar que el tamaño de la mayor de las dos sub-tablas ($S_1$ o $S_2$) sea una fracción de la tabla inicial $S$ y no sólo unos pocos elementos más pequeña.
Con el siguiente ejemplo vamos a mostrar que el algoritmo QSelect5 garantiza que las dos sub-tablas permanecen equilibradas en toda partición que efectúe  (Weiss Chapter 10).

<img src="./QSelect5.png"   width="600" height="600"  />

Por simplicidad supondremos que el número de grupos formados sea $2\cdot W+1$ donde $W$ es un entero. Por ejemplo si W=7, tendríamos 15 grupos y como cada grupo tiene 5 elementos, el número de elementos de la tabla sería $|S|=10 \cdot 15 + 5 = 155$

En la figura anterior se ha representado una tabla $S$ con 45 elementos. Cada de las columnas representa un grupo de 5 elementos. Por tanto, en este ejemplo el número de grupos $ \frac{45}{5} = 9 = 2 \cdot W + 1 $ con $W=2$.

* La fila etiquetada con *Medians* representa a las medianas de los grupos. El elemento marcado con $\nu$ es la *mediana de las medianas*.
Los elementos etiquetados con L (*Long*) representan a las medianas de los grupos con un valor mayor que $\nu$. 
Los S (*Small*) son medianas con valor menor que $\nu$. 
Los elementos marcados con H (*Hugh*) tienen un valor mayor que $\nu$. Son elementos que pertenecen a un grupo cuya mediana es mayor que $\nu$ y que, además, tienen un valor mayor que la mediana de su grupo.
Sin embargo, los elementos marcados con T (*Tiny*) son inferiores a $\nu$. 
Pertenecen a grupos cuya mediana es inferior a $\nu$ y, además, su valor es inferior a la mediana de su grupo.
Del resto de elementos, en blanco en la figura, no podemos saber si tienen un valor menor o mayor que $\nu$.

A continuación demostraremos que cuando se elija $\nu$ como pivote de una partición, las sub-listas $S_1$ y $S_2$ de $S$ **necesariamente** estarán equilibradas. Vamos a demostrar que el tamaño máximo de la mayor de las sub-listas será  inferior a $7W+3 < 0.7 \cdot n$ y, en consecuencia , el tamaño de la otra sub-lista será superior a $0.3 \cdot n$. Observa,
 * El total de elementos L, mayores que $\nu$,  es $W$, i.e. tantos como el número de grupos que tengan una mediana mayor que $\nu$.
 * EL total de elementos H, mayores que $\nu$, es $2W + 2$ (i.e., dos elementos por cada una de los grupos que tengan una mediana mayor que $\nu$, mas los dos elementos del grupo que tiene a $\nu$ por mediana).
 * Por tanto el total de elementos que *con certeza* son mayores que  $\nu$  son $L+H = 3W+2$ 
 * Igualmente el total de elementos que *con certeza* son menores que $\nu$ son $S+T = 3W+2$
 * El total de *posibles* elementos mayores que $\nu$ serán $(10W + 5) - (3W+2) = 7W  +3 < 0.7 \cdot n = \frac{7}{10}n$  


**Ley de recurrencia:**

Debemos obtener una ley de recurrencia en el peor de los casos. Sea $T(n)$ es el coste del algoritmo en el peor de los casos, entonces,

* Dividir la tabla $S$ en grupos tendrá un coste $O(n)$
* Obtener las medias de los grupos tendrá un coste $O(n) =O(1) \cdot O(n)$, ya que la mediana de cada grupo  es $O(1)$ y hay que iterar sobre los $n$ grupos.
* Obtener la mediana de las medianas $T(\lfloor n/5 \rfloor)$ ya que hay $\lfloor n/5 \rfloor$ medianas
* Particionar la tabla en las dos subtablas tendrá un coste $O(n)$
* Ejecutar la recursión en el peor de los casos, i.e., en la sub-tabla de mayor tamaño, $T( \frac{7}{10} n)$

Por tanto
$$
T(n) \leq  T \Big(\lfloor n/5 \rfloor \Big) + T \Big( \frac{7}{10} n \Big) + O(n)
$$


**Resolviendo la ley de recurrencia:**

Vamos a resolver la recurrencia anterior por el **método de substitución**: Asumimos que existe una constante $c$ tal que $T(n) \leq cn$ para un valor suficientemente grande de $n$. Utilizaremos un argumento inductivo. Como el procedimiento QSelect5 siempre finaliza (lo asumimos aunque **no** lo hemos demostrado) podemos escribir

$$
T(n)= 
\begin{cases}
O(1)  & \text{si } \; n \leq n_0 \\ 
 T \Big(\lfloor n/5 \rfloor \Big) + T \Big( \frac{7}{10} n \Big) + O(n)       & \text{para  todo } n 
\end{cases}
$$

Es decir deberán existir dos constantes $a$ y $b$ tales que $T(n)\leq b $ para $n < n_0$ y, además, 
$T(n) \leq T \Big(\lfloor n/5 \rfloor \Big) + T \Big( \frac{7}{10} n \Big) + an $ para todo $n$.
El valor de $n_0$ se elegirá mas adelante para asegurar la continuidad de la ley de recurrrencia (Cormen et al.). Aplicaremos el método inductivo para determinar el valor de $c$. Empezamos por el **caso base** en el que $n=n_0$

* **Caso base**
\begin{aligned}
T(n_0) & \leq T(n_0/5) + T (7 \cdot n_0 / 5) + a \cdot n_0 \\
    & = b + b + a \cdot n_0 \\
    & = 2 b +  n_0 a 
\end{aligned}
Entonces el caso base será cierto si $2\cdot b + n_0 \cdot a \leq c \cdot n_0$ y, por tanto, si $c \geq a + \frac{2 b}{n_0}$.

* **Caso inductivo**
Supongamos $n \geq n_0$, entonces la hipótesis inductiva debe verificar 
\begin{aligned}
T(n) \leq c & \big(\lfloor n/5 \rfloor \big) + c \Big(\frac{7}{10} \Big) n  + an \\
  \leq & \,  c \, \Big(\frac{n}{5} + 1 \Big) +  c \, \Big(\frac{7}{10} \Big) n + an \\
  = & cn + \left( - \frac{4}{5} c n+ c +  \frac{7}{10} c n+ an \right) \\
  = & c\, n + \underbrace{c - c \,  n \frac{1}{10} + a \, n }
\end{aligned}
 * Como queremos que $T(n) \leq c \, n$, es nuesta asunción, necesitamos que la parte englobada por la llave sea **no** positiva para todo $n \geq n_o$. Para ello $c \geq \frac{ 10 \, a\, n}{n-10}$. 
 * Fíjate como el segundo miembro de la desigualdad anterior esta acotado por abajo, $\Omega(1)$, siempre que  $n>10$. Por ejemplo si elegimos $n_0 = 20$ (en realidad cualquier valor de $n_0 > 10$ nos valdría) entonces  $\frac{n}{n-10} \leq 2$ para todo $n \geq 20$.
Es decir, que la parte englobada por la llave será no positiva siempre que $c \geq 20a$. 
 * Para que se satisfagan las restricciones del caso base y el caso inductivo $c \geq \text{max}\{ 20  \, a,  a + \frac{ b}{10} \}$, donde ya hemos considerado que $n_0 = 20$.  
 * **Por tanto hemos demostrado que nuestra asunción es válida. Por tanto, en el peor de los casos el coste de nuestro algoritmo es $O(n)$** 
  

# Apéndice: Coste de los algoritmos recursivos

(**Importante**) _Es importante que repases lo aprendido en la asignatura de Análisis de Algoritmos, aquí tan solo se hace un rápido repaso no muy riguroso_.

Un algoritmo típico *divide y vencerás* divide un problema dado en $a$ partes más pequeñas, cada una de las cuales es de tamaño $n/b$. Luego dedica $f(n)$ tiempo a combinar estas soluciones de subproblemas en un resultado completo. Sea $T(n)$ el tiempo en el peor de los casos que este algoritmo tarda en resolver un problema de tamaño $n$. Entonces $T(n)$ viene dada por la siguiente relación de recurrencia:
$$
T(n)= a T(n/b) + f(n);
$$

Existen varias técnicas para analizar el coste de los algoritmos recursivos. Por ejemplo
* El método de substitución
* El árbol de recurrencias
* El métodos maestro (*the master method*)
* Método de Akri-Bazi

Estas técnicas se estudian en la asignatura de análisis de algoritmos. A continuación repasamos el *método maestro*.

## El método maestro

**The master method** es el método más fácil, cuando se puede aplicar. Proporciona límites para las recurrencias de la forma 
$$
T(n)= a T(n/b) + f(n);
$$
donde $a > 0$ y $b > 1$ son constantes y $f(n)$ es una función determinada. Este tipo de recurrencia tiende a surgir con mucha frecuencia en el estudio de algoritmos recursivos. Por ejemplo, aparece en los algoritmo de *divide y vencerás* que crea subproblemas, cada uno de los cuales es $1/b$ veces el tamaño del problema original e incluye una función de coste $f(n)$  para los pasos de dividir y/o combinar. 
Para aplicar el método maestro, se deben memorizar tres posibles casos, pero una vez que lo hagas, se pueden determinar fácilmente los límites asintóticos en los tiempos de ejecución de muchos de los algoritmo tipo *divide y vencerás*.

A continuación presentemos una versión simplificada del teorema principal del *master mehtod* (Weiss et al.). Para una versión mas general consultar el Cormen. et al.

**Teorema 1**

La solución para la ley de recurrencia $T(N) = a T(n/b) + \Theta(n^k)$ con $a\geq 1$ y $b>1$


$$
T(n)= 
\begin{cases}
O(n^{\log_b a})  & \text{si } \; a > b^k \\ 
O(n^k \log n)    & \text{si } \;  a = b^k \\
O(n^k )          & \text{si }  \; a < b^k 
\end{cases}
$$

Algunos ejemplos de aplicación del teorema anterior:

* *Mergesort*: el tiempo de ejecución de mergesort se rige por la recurrencia $T (n) = 2 \, T(n/2) + O(n)$, ya que el algoritmo divide los datos en dos mitades de igual tamaño y después debe fusionar los datos con un coste lineal.  Aplicando el teorema anterior, esta recurrencia se evalúa como $T(n)=O(n \cdot \log n)$

* *Búsqueda binaria*: el tiempo de ejecución de la búsqueda binaria se rige por la recurrencia $T(n)=T(n/2) +O(1)$, ya que en cada paso dedicamos un tiempo constante para reducir el problema a una instancia de la mitad de su tamaño. Esta recurrencia se evalúa como $T(n)=O(\log n)$.

**Teorema 2**

La solución para la ley de recurrencia $T(N) = a T(n/b) + \Theta(n^k \log^p n)$ con $a\geq 1$, $b>1$ y $p\geq 0$


$$
T(n)= 
\begin{cases}
O(n^{\log_b a})  & \text{si } \;  a > b^k \\ 
O(n^k \log^{p+1} n)    & \text{si } \;  a = b^k \\
O(n^k  \log^p n )          & \text{si } \;  a < b^k 
\end{cases}
$$